Steps for building a simple agent:

- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input

In [1]:
from langchain.chat_models import ChatOpenAI

In [2]:
# setup llm
llm = ChatOpenAI(temperature=0)

In [5]:
# define tool
from langchain.tools import tool
import os
import nbformat

In [28]:
@tool
def turn_jupyter_notebook_into_python_script(jupyter_notebook_path: str) -> None:
        """Turn a jupyter notebook into a python script or a text file"""
        # Load the notebook
        output_file_path="./output_notebook.txt" 
        file_type='txt'
        with open(jupyter_notebook_path) as f:
            nb = nbformat.read(f, as_version=4)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
        
        # Convert the notebook to a Python script or a text file
        if file_type == 'py':
            nbformat.write(nb, output_file_path)
        elif file_type == 'txt':
            with open(output_file_path, 'w') as f:
                for cell in nb.cells:
                    if cell.cell_type == 'code':
                        f.write(cell.source + '\n\n')
                    elif cell.cell_type == 'markdown':
                        f.write(cell.source + '\n\n')
        else:
            raise ValueError('Invalid file type. Must be "py" or "txt".')
        
        
tools = [turn_jupyter_notebook_into_python_script]

In [24]:
# setup prompt template

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant that helps me write reports for machine learning related tasks."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [30]:
from langchain.tools.render import format_tool_to_openai_function

In [32]:
# connect llm with your tools

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [33]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [34]:
agent = {"input": lambda x: x["input"],
         "agent_scratchpad": lambda x: format_to_openai_functions(x["intermediate_steps"]),
         } | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser() 

In [39]:
agent.invoke({
    "input": "I want to write a report from this notebook I've written here: ./reporting_agent_langchain.ipynb",
    "intermediate_steps": []})

AgentActionMessageLog(tool='turn_jupyter_notebook_into_python_script', tool_input={'jupyter_notebook_path': './reporting_agent_langchain.ipynb'}, log="\nInvoking: `turn_jupyter_notebook_into_python_script` with `{'jupyter_notebook_path': './reporting_agent_langchain.ipynb'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'turn_jupyter_notebook_into_python_script', 'arguments': '{\n  "jupyter_notebook_path": "./reporting_agent_langchain.ipynb"\n}'}})])

In [36]:
from langchain.schema.agent import AgentFinish

In [40]:
intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "I want to write a report from this notebook I've written here: ./reporting_agent_langchain.ipynb",
        "intermediate_steps": intermediate_steps
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        tool = {
            "turn_jupyter_notebook_into_python_script": turn_jupyter_notebook_into_python_script
        }[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

turn_jupyter_notebook_into_python_script {'jupyter_notebook_path': './reporting_agent_langchain.ipynb'}
I apologize, but it seems that there was an error in converting the Jupyter notebook into a Python script. Could you please double-check the path to the notebook file and ensure that it is correct?
